# Generatív hálózatok

A Rekurrens Neurális Hálózatok (RNN-ek) és azok kapuzott cellaváltozatai, mint például a Hosszú Rövid Távú Memóriacellák (LSTM-ek) és a Kapuzott Rekurrens Egységek (GRU-k), lehetőséget nyújtanak a nyelvi modellezésre, azaz képesek megtanulni a szavak sorrendjét, és előrejelzéseket adni a következő szóra egy sorozatban. Ez lehetővé teszi, hogy az RNN-eket **generatív feladatokra** használjuk, például egyszerű szöveggenerálásra, gépi fordításra, sőt akár képfeliratozásra is.

Az előző egységben tárgyalt RNN architektúrában minden RNN egység a következő rejtett állapotot adta ki eredményként. Azonban hozzáadhatunk egy másik kimenetet is minden rekurrens egységhez, amely lehetővé teszi, hogy egy **sorozatot** adjunk ki (amely megegyezik az eredeti sorozat hosszával). Továbbá használhatunk olyan RNN egységeket is, amelyek nem fogadnak bemenetet minden lépésnél, hanem csak egy kezdeti állapotvektort vesznek, és ezután egy kimeneti sorozatot generálnak.

Ebben a jegyzetfüzetben egyszerű generatív modellekre fogunk összpontosítani, amelyek segítenek szöveget generálni. Az egyszerűség kedvéért építsünk egy **karakter-szintű hálózatot**, amely betűről betűre generál szöveget. Az edzés során szükségünk lesz egy szövegkorpuszra, amelyet betűsorozatokra bontunk.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset,test_dataset,classes,vocab = load_dataset()

Loading dataset...
Building vocab...


## Karakter szókincs létrehozása

Ahhoz, hogy karakter szintű generatív hálózatot építsünk, a szöveget szavak helyett egyedi karakterekre kell bontani. Ezt egy másik tokenizáló definiálásával érhetjük el:


In [2]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(char_tokenizer(line))
vocab = torchtext.vocab.vocab(counter)

vocab_size = len(vocab)
print(f"Vocabulary size = {vocab_size}")
print(f"Encoding of 'a' is {vocab.get_stoi()['a']}")
print(f"Character with code 13 is {vocab.get_itos()[13]}")

Vocabulary size = 82
Encoding of 'a' is 1
Character with code 13 is c


Lássuk a példát arra, hogyan kódolhatjuk a szöveget az adatállományunkból:


In [3]:
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

enc(train_dataset[0][1])

tensor([ 0,  1,  2,  2,  3,  4,  5,  6,  3,  7,  8,  1,  9, 10,  3, 11,  2,  1,
        12,  3,  7,  1, 13, 14,  3, 15, 16,  5, 17,  3,  5, 18,  8,  3,  7,  2,
         1, 13, 14,  3, 19, 20,  8, 21,  5,  8,  9, 10, 22,  3, 20,  8, 21,  5,
         8,  9, 10,  3, 23,  3,  4, 18, 17,  9,  5, 23, 10,  8,  2,  2,  8,  9,
        10, 24,  3,  0,  1,  2,  2,  3,  4,  5,  9,  8,  8,  5, 25, 10,  3, 26,
        12, 27, 16, 26,  2, 27, 16, 28, 29, 30,  1, 16, 26,  3, 17, 31,  3, 21,
         2,  5,  9,  1, 23, 13, 32, 16, 27, 13, 10, 24,  3,  1,  9,  8,  3, 10,
         8,  8, 27, 16, 28,  3, 28,  9,  8,  8, 16,  3,  1, 28,  1, 27, 16,  6])

## Generatív RNN tanítása

Az RNN szöveg generálására való tanításának módja a következő. Minden lépésben veszünk egy `nchars` hosszúságú karakterláncot, és megkérjük a hálózatot, hogy minden bemeneti karakterhez generálja a következő kimeneti karaktert:

![Kép, amely az 'HELLO' szó RNN általi generálását mutatja.](../../../../../translated_images/hu/rnn-generate.56c54afb52f9781d.webp)

A konkrét helyzettől függően előfordulhat, hogy speciális karaktereket is be kell vonnunk, például *sorvége* `<eos>`. A mi esetünkben azonban csak végtelen szöveg generálására szeretnénk tanítani a hálózatot, ezért minden szekvencia méretét fixen `nchars` tokenre állítjuk. Ennek megfelelően minden tanítási példában `nchars` bemenet és `nchars` kimenet lesz (a bemeneti szekvencia egy szimbólummal balra eltolva). Egy minibatch több ilyen szekvenciából fog állni.

A minibatch-ek generálásának módja az, hogy minden `l` hosszúságú hírszövegből előállítjuk az összes lehetséges bemenet-kimenet kombinációt (ezek száma `l-nchars` lesz). Ezek egy minibatch-et alkotnak, és a minibatch-ek mérete minden tanítási lépésnél eltérő lesz.


In [4]:
nchars = 100

def get_batch(s,nchars=nchars):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

get_batch(train_dataset[0][1])

(tensor([[ 0,  1,  2,  ..., 28, 29, 30],
         [ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         ...,
         [20,  8, 21,  ...,  1, 28,  1],
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16]]),
 tensor([[ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         [ 2,  3,  4,  ...,  1, 16, 26],
         ...,
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16],
         [ 5,  8,  9,  ..., 27, 16,  6]]))

Most nézzük meg a generátorhálózat meghatározását. Ez bármelyik ismétlődő cellán alapulhat, amelyet az előző egységben tárgyaltunk (egyszerű, LSTM vagy GRU). Példánkban LSTM-et fogunk használni.

Mivel a hálózat karaktereket kap bemenetként, és a szókincs mérete viszonylag kicsi, nincs szükség beágyazási rétegre, az egy-hot-kódolt bemenet közvetlenül az LSTM cellába kerülhet. Azonban, mivel karakterek számát adjuk meg bemenetként, ezeket egy-hot-kódolni kell, mielőtt az LSTM-hez továbbítanánk. Ezt a `one_hot` függvény meghívásával végezzük el a `forward` lépés során. A kimeneti kódoló egy lineáris réteg lesz, amely az elrejtett állapotot egy-hot-kódolt kimenetté alakítja.


In [5]:
class LSTMGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.rnn = torch.nn.LSTM(vocab_size,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, s=None):
        x = torch.nn.functional.one_hot(x,vocab_size).to(torch.float32)
        x,s = self.rnn(x,s)
        return self.fc(x),s

A képzés során szeretnénk képesek lenni generált szövegeket mintázni. Ehhez definiálni fogjuk a `generate` függvényt, amely egy `size` hosszúságú kimeneti sztringet állít elő, az `start` kezdeti sztringből kiindulva.

A működése a következőképpen zajlik. Először az egész kezdő sztringet átadjuk a hálózaton, majd megkapjuk az `s` kimeneti állapotot és a következő előrejelzett karaktert, `out`. Mivel az `out` egy one-hot kódolt érték, az `argmax` segítségével meghatározzuk a karakter indexét, `nc`, a szókészletben, majd az `itos` segítségével kiderítjük a tényleges karaktert, és hozzáadjuk az eredményként kapott karakterek listájához, `chars`. Ezt a karaktergenerálási folyamatot `size` alkalommal megismételjük, hogy előállítsuk a szükséges számú karaktert.


In [8]:
def generate(net,size=100,start='today '):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            nc = torch.argmax(out[0][-1])
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)

Most kezdjük el a tanulást! A tanulási ciklus szinte ugyanaz, mint az összes korábbi példánkban, de az pontosság helyett minden 1000 epoch után mintavételezett generált szöveget nyomtatunk ki.

Különös figyelmet kell fordítani arra, hogyan számítjuk ki a veszteséget. A veszteséget az egy-hot-kódolt kimenet `out` és a várt szöveg `text_out` alapján kell kiszámítani, amely a karakterindexek listája. Szerencsére a `cross_entropy` függvény az első argumentumként a nem normalizált hálózati kimenetet, a második argumentumként pedig az osztályszámot várja, ami pontosan az, amivel rendelkezünk. Emellett automatikusan átlagol a minibatch méretén.

A tanulást a `samples_to_train` minták számával korlátozzuk, hogy ne kelljen túl sokáig várni. Bátorítunk, hogy kísérletezz és próbálj ki hosszabb tanulást, akár több epochon keresztül is (ebben az esetben egy másik ciklust kellene létrehoznod ehhez a kódhoz).


In [9]:
net = LSTMGenerator(vocab_size,64).to(device)

samples_to_train = 10000
optimizer = torch.optim.Adam(net.parameters(),0.01)
loss_fn = torch.nn.CrossEntropyLoss()
net.train()
for i,x in enumerate(train_dataset):
    # x[0] is class label, x[1] is text
    if len(x[1])-nchars<10:
        continue
    samples_to_train-=1
    if not samples_to_train: break
    text_in, text_out = get_batch(x[1])
    optimizer.zero_grad()
    out,s = net(text_in)
    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
    loss.backward()
    optimizer.step()
    if i%1000==0:
        print(f"Current loss = {loss.item()}")
        print(generate(net))

Current loss = 4.398899078369141
today sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr s
Current loss = 2.161320447921753
today and to the tor to to the tor to to the tor to to the tor to to the tor to to the tor to to the tor t
Current loss = 1.6722588539123535
today and the court to the could to the could to the could to the could to the could to the could to the c
Current loss = 2.423795223236084
today and a second to the conternation of the conternation of the conternation of the conternation of the 
Current loss = 1.702607274055481
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.692358136177063
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.9722288846969604
today and the control the control the control the control the control the control the control the control 
Current loss = 1.8

Ez a példa már most is elég jó szöveget generál, de több módon is tovább lehet javítani:

* **Jobb minibatch generálás**. Az adatokat úgy készítettük elő a tanításhoz, hogy egy mintából egy minibatch-et generáltunk. Ez nem ideális, mert a minibatch-ek mérete eltérő, és néhány esetben nem is lehet őket létrehozni, mivel a szöveg kisebb, mint `nchars`. Ráadásul a kis minibatch-ek nem terhelik eléggé a GPU-t. Ésszerűbb lenne egy nagy szövegrészt venni az összes mintából, majd létrehozni az összes input-output párt, összekeverni őket, és egyenlő méretű minibatch-eket generálni.

* **Többrétegű LSTM**. Érdemes kipróbálni 2 vagy 3 rétegű LSTM cellákat. Ahogy az előző egységben említettük, az LSTM minden rétege bizonyos mintázatokat von ki a szövegből, és karakter szintű generátor esetén várható, hogy az alacsonyabb LSTM szint a szótagokért felel, míg a magasabb szintek a szavakért és szókapcsolatokért. Ezt egyszerűen megvalósíthatjuk azzal, hogy a rétegek számát paraméterként adjuk át az LSTM konstruktorának.

* Érdemes lehet kísérletezni **GRU egységekkel**, hogy megnézzük, melyik teljesít jobban, valamint **különböző rejtett réteg méretekkel**. Túl nagy rejtett réteg túltanuláshoz vezethet (például a hálózat megtanulja a szöveget szó szerint), míg a kisebb méret nem biztos, hogy jó eredményt hoz.


## Lágy szöveg generálás és hőmérséklet

Az előző `generate` definícióban mindig a legnagyobb valószínűségű karaktert választottuk ki következő karakterként a generált szövegben. Ennek eredményeként a szöveg gyakran "ismételte" ugyanazokat a karakter-szekvenciákat újra és újra, mint ebben a példában:
```
today of the second the company and a second the company ...
```

Azonban, ha megnézzük a következő karakter valószínűségi eloszlását, előfordulhat, hogy a legnagyobb valószínűségek közötti különbség nem túl nagy, például egy karakter valószínűsége lehet 0.2, míg egy másiké 0.19, stb. Például, ha a '*play*' szekvencia következő karakterét keressük, a következő karakter lehet egyaránt szóköz vagy **e** (mint a *player* szóban).

Ez arra a következtetésre vezet minket, hogy nem mindig "igazságos" a magasabb valószínűségű karaktert választani, mert a második legnagyobb valószínűségű karakter választása is értelmes szöveghez vezethet. Bölcsebb, ha **mintát veszünk** a hálózat kimenete által adott valószínűségi eloszlásból.

Ez a mintavétel a `multinomial` függvénnyel végezhető el, amely az úgynevezett **multinomiális eloszlást** valósítja meg. Az alábbiakban definiálva van egy függvény, amely ezt a **lágy** szöveg generálást valósítja meg:


In [10]:
def generate_soft(net,size=100,start='today ',temperature=1.0):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            #nc = torch.argmax(out[0][-1])
            out_dist = out[0][-1].div(temperature).exp()
            nc = torch.multinomial(out_dist,1)[0]
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start='Today ',temperature=i)}\n")

--- Temperature = 0.3
Today and a company and complete an all the land the restrational the as a security and has provers the pay to and a report and the computer in the stand has filities and working the law the stations for a company and with the company and the final the first company and refight of the state and and workin

--- Temperature = 0.8
Today he oniis its first to Aus bomblaties the marmation a to manan  boogot that pirate assaid a relaid their that goverfin the the Cappets Ecrotional Assonia Cition targets it annight the w scyments Blamity #39;s TVeer Diercheg Reserals fran envyuil that of ster said access what succers of Dour-provelith

--- Temperature = 1.0
Today holy they a 11 will meda a toket subsuaties, engins for Chanos, they's has stainger past to opening orital his thempting new Nattona was al innerforder advan-than #36;s night year his religuled talitatian what the but with Wednesday to Justment will wemen of Mark CCC Camp as Timed Nae wome a leaders

--- Temper

Bevezettünk egy újabb paramétert, amelyet **hőmérsékletnek** nevezünk, és amely azt jelzi, mennyire ragaszkodjunk a legmagasabb valószínűséghez. Ha a hőmérséklet 1,0, akkor tisztességes multinomiális mintavételt végzünk, és amikor a hőmérséklet végtelenre nő - minden valószínűség egyenlővé válik, és véletlenszerűen választjuk ki a következő karaktert. Az alábbi példában megfigyelhetjük, hogy a szöveg értelmetlenné válik, amikor túlzottan növeljük a hőmérsékletet, és "ciklusos", keményen generált szövegre hasonlít, amikor közelebb kerül a 0-hoz.



---

**Felelősség kizárása**:  
Ez a dokumentum az AI fordítási szolgáltatás, a [Co-op Translator](https://github.com/Azure/co-op-translator) segítségével lett lefordítva. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Kritikus információk esetén javasolt professzionális emberi fordítást igénybe venni. Nem vállalunk felelősséget semmilyen félreértésért vagy téves értelmezésért, amely a fordítás használatából eredhet.
